## MuseScore Scraper

Tool: https://github.com/frankye8998/MusicalMusic  

[Reference 2](https://github.com/codeandproduce/music_research_dataset_midi/blob/9a5afe86f1f2d3fdbfa24de99122140e7a7cd60c/MuseScore/MuseScore%20Scraping%20Main.ipynb)

In [3]:
from lxml import html
import urllib
import requests
import shutil

In [4]:
from random import randint
from time import sleep

In [5]:
import json
from pathlib import Path
from tqdm import tqdm
import time

In [6]:
# links = []
# url = "https://musescore.com/hub/piano/solo-piano?sort=view_count"
# # url = "https://musescore.com/hub/piano/piano-duet?sort=view_count"
# page = requests.get(url)
# tree = html.fromstring(page.content)
# print(tree.xpath("//title/text()"))

In [7]:
# articles = tree.xpath("//article[@role='article']")

In [8]:
def get_data(a):
    score = a.find("h2//a[@rel='bookmark']")
    link = score.attrib['href']
    title = score.text_content().strip()
    author = a.find("div[@class='user']//a").text_content()
    metadata = a.find("div[@class='meta']").text_content().split('•\n')
    parts, pages, duration, time, views = [m.strip() for m in metadata]
    return {
        'link': "https://musescore.com" + link,
        'score_id': Path(link).name,
        'title': title,
        'author': author,
        'parts': parts,
        'pages': pages,
        'duration': duration,
        'views': views
    }

## But that's actually just page one and there are pages 2,3,4,5... 100!
### So let's make a more general format:

In [24]:
# url_default = "https://musescore.com/hub/video_games/movie?sort=view_count&page=" # + the page number
# url_default = 'https://musescore.com/hub/video_games?instruments=0&sort=view_count&page='
# url_default = 'https://musescore.com/hub/video_games/easy?instruments=0&sort=view_count&page='
# url_default = 'https://musescore.com/hub/video_games/game?instruments=0&sort=view_count&page='
url_default = 'https://musescore.com/hub/piano/piano-duet?instruments=0&sort=view_count&page='
# url_default = 'https://musescore.com/hub/video_games/anime?instruments=0&sort=view_count&page='
# url_default = "https://musescore.com/hub/piano/solo-piano?sort=view_count&page=" # + the page number
# url_default = "https://musescore.com/hub/piano/voice-piano?sort=view_count&page=" # + the page number
# url_default = "https://musescore.com/hub/piano?sort=view_count&page=" # + the page number

In [25]:
json_file = Path('musescore_duet.json')

In [22]:
if json_file.exists():
    with open(json_file, 'r') as fp:
        links = json.load(fp)
        

In [27]:
if not json_file.exists():
    links = []
    for page in tqdm(range(1,100), total=100):
        page = requests.get(url_default+str(page)) # https://musescore.com/hub/piano?page=1,2,3,4,5,...,10
        tree = html.fromstring(page.content)

        articles = tree.xpath("//article[@role='article']")

        links.extend([get_data(a) for a in articles])

#         sleep(randint(1,4))
    with open(json_file, 'w') as fp:
        json.dump(links, fp)
print(links[:100])

 99%|█████████▉| 99/100 [01:23<00:00,  1.53it/s]

[{'link': 'https://musescore.com/user/73972/scores/1352796', 'score_id': '1352796', 'title': 'Undertale - Megalovania (Piano) [Added guitar, fixed tonality]', 'author': 'Manel Návola', 'parts': '2 parts', 'pages': '5 pages', 'duration': '02:30', 'views': '520,559 views'}, {'link': 'https://musescore.com/user/10363416/scores/3332996', 'score_id': '3332996', 'title': 'All Of Me - John Legend (Piano Cover - ReiK)', 'author': 'ReiKGaminG', 'parts': '2 parts', 'pages': '6 pages', 'duration': '04:57', 'views': '366,192 views'}, {'link': 'https://musescore.com/user/2400376/scores/1980426', 'score_id': '1980426', 'title': 'Pirates of the Caribbean - Easy Piano', 'author': 'Niall Devlin', 'parts': '2 parts', 'pages': '3 pages', 'duration': '02:10', 'views': '289,193 views'}, {'link': 'https://musescore.com/user/5449711/scores/2221711', 'score_id': '2221711', 'title': 'Piano Man (Piano)', 'author': 'williamcreamer2', 'parts': '2 parts', 'pages': '16 pages', 'duration': '06:03', 'views': '236,641

# Scrape links found

### Musical Music Lib

In [11]:

class MuseScoreException(Exception):
    pass

class InvalidFileExtension(MuseScoreException):
    pass

class InvalidScoreID(MuseScoreException):
    pass

class InvalidCredentials(MuseScoreException):
    pass

class InvalidSearchSort(MuseScoreException):
    pass

In [12]:
import urllib
import urllib.request


import bs4
import requests

def urlretrieve(url: str, fn: Path, timeout=timeout):
    with fn.open('wb') as f:
        f.write(requests.get(url, allow_redirects=True, timeout=timeout).content)
        
class MusicalMusic:
    """Musescore actions requiring an account."""

    def __init__(self, username, password):
        self.username = username
        url = "https://musescore.com/user/login"
        r = requests.get(url)
        soup = bs4.BeautifulSoup(r.text, "html.parser")
        csrf = soup.find("meta", {"name": "csrf-token"})["content"]
        url = "https://musescore.com/user/auth/login/process"
        cookies = {
            "mu_browser_uni": r.cookies['mu_browser_uni'],
            "_csrf": r.cookies["_csrf"]
        }
        data = {
          "username": username,
          "password": password,
          "_csrf": csrf,
          "op": "Log in"
        }
        try:
            mu_user = requests.post(url,
                                    data=data,
                                    cookies=cookies,
                                    allow_redirects=False,
                                    ).cookies["mu_user_new"]
        except KeyError as e:
            raise InvalidCredentials(
                "Please check your username and password!") from e

        mu_browser_uni = r.cookies['mu_browser_uni']

        self.mu_browser_uni = mu_browser_uni
        self.mu_user = mu_user

    def retrieve(self, id, format="pdf"):
        """Retrieves Musescore data in bytes"""

        if format not in ["mp3", "pdf", "mid", "mxl", "mscz"]:
            raise InvalidFileExtension("Must be mp3, pdf, mid, mxl, or mscz.")
        newlink = f"https://musescore.com/score/{id}/download/{format}"
        cookies = {"mu_browser_uni": self.mu_browser_uni,
                   "mu_user_new": self.mu_user}
        bytes = requests.get(newlink, cookies=cookies, verify=False)
        if bytes.status_code != 200:
            raise InvalidScoreID(str(bytes.status_code))
        return bytes.content

    def download(self, id, filename, format="mp3", proxy=None):
        if format not in ["mp3", "pdf", "mid", "mxl", "mscz"]:
            raise InvalidFileExtension("Must be mp3, pdf, mid, mxl, or mscz.")
        newlink = f"https://musescore.com/score/{id}/download/{format}"
        if proxy:
            proxy_support = urllib.request.ProxyHandler({'https' : proxy})
            opener = urllib.request.build_opener(proxy_support)
        else: opener = urllib.request.build_opener()
            
        cookieString = f"mu_browser_uni={self.mu_browser_uni};" \
                        f"mu_user_new={self.mu_user}"
        opener.addheaders = [("cookie", cookieString)]
        urllib.request.install_opener(opener)
        try:
            urllib.request.urlretrieve(newlink, filename)
        except urllib.error.HTTPError as e:
            raise e


### Trying proxy

https://codelike.pro/create-a-crawler-with-rotating-ip-proxy-in-python/

In [13]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import random

ua = UserAgent() # From here we generate a random user agent
proxies = [] # Will contain proxies [ip, port]

In [14]:
# Main function
  # Retrieve latest proxies
proxies_req = Request('https://www.sslproxies.org/')
proxies_req.add_header('User-Agent', ua.random)
proxies_doc = urlopen(proxies_req).read().decode('utf8')

soup = BeautifulSoup(proxies_doc, 'html.parser')
proxies_table = soup.find(id='proxylisttable')

# Save proxies in the array
for row in proxies_table.tbody.find_all('tr'):
    proxies.append({
      'ip':   row.find_all('td')[0].string,
      'port': row.find_all('td')[1].string
    })

### More proxies

https://github.com/constverum/ProxyBroker

In [60]:
import asyncio
from proxybroker import Broker

more_proxies = []

async def show(proxy_queue):
    while True:
        proxy = await proxy_queue.get()
        if proxy is None: break
        print('Found proxy: %s' % proxy)
        more_proxies.append({ 'ip': proxy.host, 'port': proxy.port })

countries = ['US', 'CA']

proxy_queue = asyncio.Queue()
broker = Broker(proxy_queue, timeout=8, max_resp_time=4)
tasks = await asyncio.gather(
    broker.find(types=['HTTPS'], countries=countries, max_resp_time=4, limit=30),
    show(proxy_queue))

Found proxy: <Proxy US 0.22s [HTTPS] 35.245.232.172:3128>
Found proxy: <Proxy US 0.23s [HTTPS] 167.99.52.107:8888>
Found proxy: <Proxy CA 0.31s [HTTPS] 158.69.138.9:1080>
Found proxy: <Proxy US 0.31s [HTTPS] 50.239.2.142:8080>
Found proxy: <Proxy US 0.54s [HTTPS] 173.249.0.209:3128>
Found proxy: <Proxy US 0.37s [HTTPS] 192.243.109.188:8080>
Found proxy: <Proxy US 0.17s [HTTPS] 184.105.143.66:3128>
Found proxy: <Proxy US 0.17s [HTTPS] 157.230.137.96:3128>
Found proxy: <Proxy US 0.17s [HTTPS] 138.197.208.221:8080>
Found proxy: <Proxy US 0.17s [HTTPS] 134.209.13.153:8080>
Found proxy: <Proxy US 0.17s [HTTPS] 157.230.150.101:8080>
Found proxy: <Proxy US 0.18s [HTTPS] 35.245.208.185:3128>
Found proxy: <Proxy CA 0.64s [HTTPS] 24.70.103.129:48002>
Found proxy: <Proxy US 0.18s [HTTPS] 68.183.103.88:8080>
Found proxy: <Proxy CA 0.19s [HTTPS] 142.93.195.94:8080>
Found proxy: <Proxy US 0.19s [HTTPS] 104.248.236.12:8080>
Found proxy: <Proxy US 0.19s [HTTPS] 3.17.175.232:3128>
Found proxy: <Proxy U

### Random proxy

In [61]:
# all_proxies = proxies + more_proxies
all_proxies = more_proxies
# Retrieve a random index proxy (we need the index to delete it if not working)
def random_proxy():
    return random.randint(0, len(all_proxies) - 1)

proxy = None
# # Choose a random proxy
# proxy_index = random_proxy()
# proxy = all_proxies[proxy_index]
# proxy_url = proxy['ip'] + ':' + str(proxy['port']); proxy_url

## Actual scraping

In [48]:
invalid_ids = []

In [31]:
deleted_proxies = []

In [19]:
with open('accounts.json', 'r') as fp:
    accounts = json.load(fp)

instances = [MusicalMusic(username, password) for (username, password) in accounts]
instance = None

In [40]:
for idx,link in enumerate(tqdm(links, total=len(links))):
    score_id = link['score_id']
    out_file = f"data/{score_id}.mxl"
    if Path(out_file).exists() or score_id in invalid_ids: continue
    # Every 10 requests, generate a new proxy
    if idx % 10 == 0 or instance is None:
        instance_index = random.randint(0, len(instances) - 1)
        instance = instances[instance_index]
    try:
        print('Downloading score id:', score_id)
        instance.download(score_id, out_file, format='mxl')
    except Exception as e:
        print('Could not download id:', score_id)
        instance = None
#     sleep(randint(1,2))
    










  0%|          | 0/1980 [00:00<?, ?it/s]










 76%|███████▋  | 1513/1980 [00:03<00:00, 490.06it/s]










 76%|███████▋  | 1513/1980 [00:16<00:00, 490.06it/s]








 77%|███████▋  | 1518/1980 [00:16<06:23,  1.21it/s] 










 77%|███████▋  | 1519/1980 [00:20<13:14,  1.72s/it]










 77%|███████▋  | 1520/1980 [00:24<17:13,  2.25s/it]

...










 82%|████████▏ | 1633/1980 [05:17<15:27,  2.67s/it]










 83%|████████▎ | 1634/1980 [05:21<18:43,  3.25s/it]










 83%|████████▎ | 1635/1980 [05:23<16:08,  2.81s/it]










 83%|████████▎ | 1636/1980 [05:26<16:29,  2.88s/it]

KeyboardInterrupt: 

In [32]:
len(all_proxies)

200

In [62]:
def download_link(link, instance, proxy):
    score_id = link['score_id']
    out_file = f"data/{score_id}.mxl"
    if Path(out_file).exists(): return False
    print('Downloading score id:', score_id)
    proxy_url = proxy['ip'] + ':' + str(proxy['port'])
    instance.download(score_id, out_file, format='mxl', proxy=proxy_url)
    return True

In [66]:
for idx,link in enumerate(tqdm(links, total=len(links))):
    proxy_index = random.randint(0, len(all_proxies) - 1)
    proxy = all_proxies[proxy_index]
    instance = instances[random.randint(0, len(instances) - 1)]
    try:
        start = time.time()
        did_download = download_link(link, instance, proxy)
#         if did_download: sleep(1)
        end = time.time()
        if (end - start) > 10: raise Exception('Took too long to download')
    except Exception as e:
        print('Could not download id:', score_id, e)
        invalid_ids.append(score_id)
        del all_proxies[proxy_index]
        print(instance.username)
        proxy = None
        instance = None

















  0%|          | 0/1980 [00:00<?, ?it/s]

















 87%|████████▋ | 1722/1980 [00:00<00:00, 1746.25it/s]

















 87%|████████▋ | 1722/1980 [00:13<00:00, 1746.25it/s]















 87%|████████▋ | 1731/1980 [00:14<01:50,  2.25it/s]  

















 87%|████████▋ | 1732/1980 [00:15<02:19,  1.78it/s]

















 88%|████████▊ | 1733/1980 [00:19<06:28,  1.57s/it]

...

















100%|█████████▉| 1976/1980 [11:02<00:07,  1.78s/it]

















100%|█████████▉| 1977/1980 [11:07<00:07,  2.57s/it]

















100%|█████████▉| 1978/1980 [11:08<00:04,  2.11s/it]

















100%|█████████▉| 1979/1980 [11:09<00:01,  1.87s/it]

















100%|██████████| 1980/1980 [11:11<00:00,  1.78s/it]

















In [64]:
del all_proxies[proxy_index]
proxy = None

### Second run

In [23]:
# Main function
  # Retrieve latest proxies
proxies_req = Request('https://www.sslproxies.org/')
proxies_req.add_header('User-Agent', ua.random)
proxies_doc = urlopen(proxies_req).read().decode('utf8')

soup = BeautifulSoup(proxies_doc, 'html.parser')
proxies_table = soup.find(id='proxylisttable')

# Save proxies in the array
for row in proxies_table.tbody.find_all('tr'):
    proxies.append({
      'ip':   row.find_all('td')[0].string,
      'port': row.find_all('td')[1].string
    })

HTTPError: HTTP Error 403: Forbidden

In [ ]:
import asyncio
from proxybroker import Broker

more_proxies = []

async def show(proxy_queue):
    while True:
        proxy = await proxy_queue.get()
        if proxy is None: break
        print('Found proxy: %s' % proxy)
        more_proxies.append({ 'ip': proxy.host, 'port': proxy.port })

proxy_queue = asyncio.Queue()
broker = Broker(proxy_queue)
tasks = await asyncio.gather(
    broker.find(types=['HTTP', 'HTTPS'], limit=400),
    show(proxy_queue))

In [ ]:
all_proxies = proxies + more_proxies
# Retrieve a random index proxy (we need the index to delete it if not working)
def random_proxy():
    return random.randint(0, len(all_proxies) - 1)

proxy = None
# # Choose a random proxy
# proxy_index = random_proxy()
# proxy = all_proxies[proxy_index]
# proxy_url = proxy['ip'] + ':' + str(proxy['port']); proxy_url

In [ ]:
json_file = Path('musescore_movie.json')

In [ ]:
if json_file.exists():
    with open(json_file, 'r') as fp:
        links = json.load(fp)
        

In [ ]:
for idx,link in enumerate(tqdm(links, total=len(links))):
    score_id = link['score_id']
    out_file = f"data/{score_id}.mxl"
    if Path(out_file).exists() or score_id in invalid_ids: continue
    # Every 10 requests, generate a new proxy
    if idx % 10 == 0 or proxy is None or instance is None:
        proxy_index = random.randint(0, len(all_proxies) - 1)
        proxy = all_proxies[proxy_index]
        proxy_url = proxy['ip'] + ':' + str(proxy['port']); proxy_url
        instance_index = random.randint(0, len(instances) - 1)
        instance = instances[instance_index]
    try:
        print('Downloading score id:', score_id)
        instance.download(score_id, out_file, format='mxl', proxy=proxy_url)
    except Exception as e:
        print('Could not download id:', score_id)
        print('Error:', e)
        invalid_ids.append(score_id)
        deleted_proxies.append(all_proxies[proxy_index])
        del all_proxies[proxy_index]
        print('Proxy ' + proxy['ip'] + ':' + str(proxy['port']) + ' deleted.')
        print(instance.username)
        proxy = None
        instance = None
    sleep(randint(1,2))
    